# Atividade 02: Ingestão e ETL com linguagem de programação (Python)

- Utilizar linguagem de programação Python para ingestão e tratamento de dados.
    - Pacotes adicionais podem ser utilizados (exceto Spark)
    - Tratamento de dados não deve ser realizado via SQL
- Realizar a ingestão de todas as base de dados em um banco de dados relacional open source. Pode ser utilizado qualquer banco de dado sendo algumas sugestões:
    - MySQL
    - Postgre
    - ClickHouse
- Gerar uma tabela final com os dados tratados e unidos.
    - O tratamento de dados deve ser realizado através da linguagem de programação Python
- Adicionar as seguintes camadas de processamento, dentro do próprio banco de dados ou em disco local. A Camada Delivery deve obrigatoriamente ter estar também no formato de uma tabela final dentro do banco de dados relacional:
    - RAW – formato dos dados livre
    - Trusted – formato de dados em Parquet ou ORC or AVRO (indicado Parquet)
    - Delivery– formato de dados em Parquet ou ORC or AVRO (indicado Parquet)

## To Do

- raw 
    - salvar em parquet (ok)
    - salvar rwzd_{tabela} (ok)
- trusted
    - ler da raw
    - aplicar data quality
        - trocar os nomes das colunas para ingles
        - trocar os nomes das colunas para snakecase
        - mudar tipos
        - outras transformacoes
            - remover string "- prudential"
            - remover coluna unnamed
- delivery
    - tabela consolidada
    - agregação

- organizar jupyter

---
## Importação Bibliotecas

In [7]:
import pandas as pd
import os

---
## Raw

### Arquivos Bancos

In [8]:
banks = pd.read_csv('./trzd/Bancos/EnquadramentoInicia_v2_alterado_manualmente_revisado.csv', sep='\t', encoding='utf8')

In [9]:
banks.head()

,Segmento,CNPJ,Nome
0,S1,0,BANCO DO BRASIL - PRUDENCIAL
1,S1,60746948,BRADESCO - PRUDENCIAL
2,S1,30306294,BTG PACTUAL - PRUDENCIAL
3,S1,360305,CAIXA ECONOMICA FEDERAL - PRUDENCIAL
4,S1,60872504,ITAU - PRUDENCIAL


In [11]:
banks.to_parquet('./raw/rwzd_banks.parquet', engine='pyarrow', compression='snappy')

### Arquivo Empregados

In [13]:
employees = pd.read_csv('./trzd/Empregados/glassdoor_consolidado_join_match_v2.csv', sep='|', encoding='utf-8')

In [14]:
employees.head(2)

,employer_name,reviews_count,culture_count,salaries_count,benefits_count,employer-website,employer-headquarters,employer-founded,employer-industry,employer-revenue,...,Diversidade e inclusão,Qualidade de vida,Alta liderança,Remuneração e benefícios,Oportunidades de carreira,Recomendam para outras pessoas(%),Perspectiva positiva da empresa(%),Segmento,Nome,match_percent
0,BNP Paribas,13000,4100,20000,3600,http://www.group.bnpparibas,"Paris, França",2000.0,/Explorar/melhores-serviços-bancários-e-de-cré...,Mais de US$ 10 bilhões,...,4.0,3.8,3.4,3.4,3.5,77.0,63.0,S3,BNP PARIBAS,100
1,BTG Pactual,1600,683,2800,635,http://www.btgpactual.com,"São Paulo, Brasil",NaN,/Explorar/melhores-gestão-de-ativos-e-investim...,Desconhecida/não se aplica,...,3.5,2.8,3.7,4.4,4.2,78.0,73.0,S1,BTG PACTUAL,100


In [16]:
employees.to_parquet('./raw/rwzd_employees.parquet', engine='pyarrow', compression='snappy')

### Arquivo Reclamações

In [17]:
list_df_claims = []
dir = './trzd/Reclamações'

for file in os.listdir(dir):
    full_path = os.path.join(dir, file)
    print(f'Processing file: {full_path}')
    list_df_claims.append(pd.read_csv(full_path, sep=';', encoding='utf-8'))


Processing file: ./trzd/Reclamações\2021_tri_01.csv
Processing file: ./trzd/Reclamações\2021_tri_02.csv
Processing file: ./trzd/Reclamações\2021_tri_03.csv
Processing file: ./trzd/Reclamações\2021_tri_04.csv
Processing file: ./trzd/Reclamações\2022_tri_01.csv
Processing file: ./trzd/Reclamações\2022_tri_03.csv
Processing file: ./trzd/Reclamações\2022_tri_04.csv


In [18]:
claims = pd.concat(list_df_claims)
claims.head(2)

,Ano,Trimestre,Categoria,Tipo,CNPJ IF,Instituição financeira,Índice,Quantidade de reclamações reguladas procedentes,Quantidade de reclamações reguladas - outras,Quantidade de reclamações não reguladas,Quantidade total de reclamações,Quantidade total de clientes – CCS e SCR,Quantidade de clientes – CCS,Quantidade de clientes – SCR,Unnamed: 14
0,2021,1º,Grupo Secundário,Conglomerado,,ABC-BRASIL (conglomerado),,2,3,4,9,26230,24698,3810,NaN
1,2021,1º,Grupo Secundário,Conglomerado,,AGIBANK (conglomerado),"54,79",58,140,73,271,1058431,790848,693843,NaN


In [43]:
# este campo estava com strings e numeros. Como haviam strings vazias, converti a coluna para str a fim de discutirmos depois como tratar na etapa Trusted
claims['Quantidade total de clientes – CCS e SCR'] = claims['Quantidade total de clientes – CCS e SCR'].astype(str)

In [44]:
claims.to_parquet('./raw/rwzd_claims.parquet', engine='pyarrow', compression='snappy')

---
## Trusted

In [63]:
# banks['Nome'] = banks['Nome'].replace(to_replace = r' - PRUDENCIAL', value = "", regex=True)

---
## Delivery

---
## Gerar arquivo python
Exemplo

In [62]:
%%writefile ./scripts/testes.pyg
print('hello world')

Writing ./scripts/testes.pyg
